In [ ]:
%%flink_gateway_sql_query
INSERT INTO
    constraint_trigger_table  WITH A1 as (
        
        SELECT
            A.id AS this,
             A.`type` as typ,
             IFNULL(A.`deleted`, false) as edeleted,
             C.`type` AS entity,
             COALESCE(E.`type`, B.`type`) AS link,
             COALESCE(E.`nodeType`, B.`nodeType`) as nodeType,
             COALESCE(E.`deleted`, B.`deleted`) as `adeleted`,
             COALESCE(E.`datasetId`, B.`datasetId`) as `index`,
             D.targetClass as targetClass,
             COALESCE(D.subpropertyPath, D.propertyPath) as propertyPath,
             CASE
                WHEN D.subpropertyPath IS NULL THEN ''
                ELSE D.propertyPath || '[' || CASE
                    WHEN B.`datasetId` = '@none' THEN '0'
                    ELSE B.`datasetId`
                END || '] ==> '
            END as parentPath,
             COALESCE(D.subpropertyPath, D.propertyPath) || '[' || CASE
                WHEN COALESCE(E.`datasetId`, B.`datasetId`) = '@none' THEN '0'
                ELSE COALESCE(E.`datasetId`, B.`datasetId`)
            END || ']' as printPath,
             D.propertyClass as propertyClass,
             D.attributeType as attributeType,
             D.maxCount as maxCount,
             D.minCount as minCount,
             D.severity as severity,
             D.id as constraint_id 
        FROM
            entities_view AS A
            JOIN constraint_table as D ON A.`type` = D.targetClass 
            LEFT JOIN attributes_view AS B ON B.name = D.propertyPath
            and B.entityId = A.id
            and B.parentId IS NULL 
            LEFT JOIN attributes_view AS E ON E.name = D.subpropertyPath
            and E.entityId = A.id
            and E.parentId = B.id 
            LEFT JOIN entities_view AS C ON COALESCE(E.`attributeValue`, B.`attributeValue`) = C.id 
        WHERE
            (
                D.subpropertyPath IS NULL
                or E.id is not NULL
            )
            and D.attributeType = 'https://uri.etsi.org/ngsi-ld/Relationship' 
    )  
SELECT
    this AS resource,
     'ClassConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN  NOT edeleted
        AND NOT IFNULL(adeleted, false)
        AND link IS NOT NULL
        AND entity IS NULL  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN  NOT edeleted
        AND NOT IFNULL(adeleted, false)
        AND link IS NOT NULL
        AND entity IS NULL  THEN `severity` 
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN  NOT edeleted
        AND NOT IFNULL(adeleted, false)
        AND link IS NOT NULL
        AND entity IS NULL   THEN 'Model validation for relationship' || `propertyPath` || 'failed for ' || this || '. Relationship not linked to existing entity of type ' || `propertyClass` || '.' 
        ELSE 'All ok'
    END as `text` 
FROM
    A1
WHERE
    A1.propertyClass IS NOT NULL
    and `index` IS NOT NULL UNION ALL  
SELECT
    this AS resource,
     'CountConstraintComponent(' || `parentPath` || `propertyPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN  NOT edeleted
        AND (
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) > TRY_CAST(`maxCount` AS INTEGER) 
            OR SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) < TRY_CAST(`minCount` AS INTEGER)
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN  NOT edeleted
        AND (
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) > TRY_CAST(`maxCount` AS INTEGER) 
            OR SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) < TRY_CAST(`minCount` AS INTEGER)
        )   THEN `severity` 
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN  NOT edeleted
        AND (
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) > TRY_CAST(`maxCount` AS INTEGER) 
            OR SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) < TRY_CAST(`minCount` AS INTEGER)
        )   THEN  'Model validation for relationship ' || `propertyPath` || 'failed for ' || this || ' . Found ' ||  TRY_CAST(
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    AND link IS NOT NULL THEN 1
                    ELSE 0
                END
            ) AS STRING
        ) || ' relationships instead of\n                            [' || `minCount` || ', ' || `maxCount` || ']!' 
        ELSE 'All ok'
    END as `text` 
FROM
    A1
WHERE
    `minCount` is NOT NULL
    or `maxCount` is NOT NULL 
group by
    this,
    edeleted,
    propertyPath,
    maxCount,
    minCount,
    severity,
    constraint_id,
    parentPath UNION ALL  
SELECT
    this AS resource,
     'NodeKindConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN  NOT edeleted
        AND NOT IFNULL(`adeleted`, false)
        AND (
            nodeType <> '@id'
            OR link <> attributeType
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN  NOT edeleted
        AND NOT IFNULL(`adeleted`, false)
        AND (
            nodeType <> '@id'
            OR link <> attributeType
        )   THEN `severity` 
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN  NOT edeleted
        AND NOT IFNULL(`adeleted`, false)
        AND (
            nodeType <> '@id'
            OR link <> attributeType
        )   THEN  'Model validation for relationship ' || `propertyPath` || ' failed for ' || this || ' . Either NodeType ' || nodeType || ' is not an IRI or type is not a Relationship.' 
        ELSE 'All ok'
    END as `text` 
FROM
    A1
WHERE
    `index` IS NOT NULL;

In [ ]:
%%flink_gateway_sql_query
INSERT INTO
    constraint_trigger_table WITH A1 AS (
        SELECT
            A.id as this,
             A.`type` as typ,
             IFNULL(A.`deleted`, false) as edeleted,
             COALESCE(E.`attributeValue`, B.`attributeValue`) as val,
             COALESCE(E.`nodeType`, B.`nodeType`) as nodeType,
             COALESCE(E.`type`, B.`type`) as attr_typ,
             COALESCE(E.`deleted`, B.`deleted`) as `adeleted`,
             COALESCE(E.`valueType`, B.`valueType`) as `valueType`,
             C.subject as foundVal,
             C.object as foundClass,
             COALESCE(E.`datasetId`, B.`datasetId`) as `index`,
             COALESCE(D.subpropertyPath, D.propertyPath) as propertyPath,
             CASE
                WHEN D.subpropertyPath IS NULL THEN ''
                ELSE D.propertyPath || '[' || CASE
                    WHEN B.`datasetId` = '@none' THEN '0'
                    ELSE B.`datasetId`
                END || '] ==> '
            END as parentPath,
             COALESCE(D.subpropertyPath, D.propertyPath) || '[' || CASE
                WHEN COALESCE(E.`datasetId`, B.`datasetId`) = '@none' THEN '0'
                ELSE COALESCE(E.`datasetId`, B.`datasetId`)
            END || ']' as printPath,
             D.propertyClass as propertyClass,
             IFNULL(D.propertyNodetype, 'null') as propertyNodetype,
             D.attributeType as attributeType,
             D.maxCount as maxCount,
             D.minCount as minCount,
             D.severity as severity,
             D.minExclusive as minExclusive,
             D.maxExclusive as maxExclusive,
             D.minInclusive as minInclusive,
             D.maxInclusive as maxInclusive,
             D.minLength as minLength,
             D.maxLength as maxLength,
             D.`pattern` as `pattern`,
             D.ins as ins,
             D.datatypes as datatypes,
             D.hasValue as hasValue,
             D.id as constraint_id 
        FROM
            `entities_view` AS A
            JOIN constraint_table as D ON A.`type` = D.targetClass 
            LEFT JOIN attributes_view AS B ON D.propertyPath = B.name
            and B.entityId = A.id
            and B.parentId IS NULL 
            LEFT JOIN attributes_view AS E ON D.subpropertyPath = E.name
            and E.entityId = A.id
            and B.id = E.parentId 
            LEFT JOIN rdf as C ON C.subject = '<' || COALESCE(E.attributeValue, B.attributeValue) || '>' 
            and C.predicate = '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
            and C.object = '<' || D.propertyClass || '>' 
        WHERE
            (
                D.subpropertyPath IS NULL
                or E.id is not NULL
            )
            and attributeType IN (
                'https://uri.etsi.org/ngsi-ld/Property',
                'https://uri.etsi.org/ngsi-ld/ListProperty',
                'https://uri.etsi.org/ngsi-ld/JsonProperty'
            ) 
    )  SELECT this AS resource,
     'NodeKindConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND NOT IFNULL(adeleted, false)
        AND (
            nodeType <> `propertyNodetype`
            OR attr_typ <> attributeType
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND NOT IFNULL(adeleted, false)
        AND (
            nodeType <> `propertyNodetype`
            OR attr_typ <> attributeType
        )  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND NOT IFNULL(adeleted, false)
        AND (
            nodeType <> `propertyNodetype`
            OR attr_typ <> attributeType
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Node is not ' ||  'of nodetype \"' || `nodeType` || '\" or not of attribute type \"' || attributeType || '\"' 
        ELSE 'All ok'
    END as `text`  FROM A1
WHERE
    propertyNodetype IS NOT NULL
    and `index` IS NOT NULL UNION ALL   SELECT this AS resource,
     'DatatypeConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            val is NULL
            OR foundVal is NULL
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            val is NULL
            OR foundVal is NULL
        )   THEN `severity` 
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            val is NULL
            OR foundVal is NULL
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Invalid value ' || IFNULL(val, 'NULL') || ' not type of ' || `propertyClass` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
WHERE
    propertyNodetype = '@id'
    and propertyClass IS NOT NULL
    and NOT IFNULL(adeleted, false)
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'MinExclusiveConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) > TRY_CAST(`minExclusive` AS DOUBLE)
            )
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) > TRY_CAST(`minExclusive` AS DOUBLE)
            )
        )  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) > TRY_CAST(`minExclusive` AS DOUBLE)
            )
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' not comparable with ' || `minExclusive` || '.' 
        WHEN typ IS NOT NULL
        AND attr_typ IS NOT NULL
        AND NOT (
            TRY_CAST(val as DOUBLE) > TRY_CAST(`minExclusive` as DOUBLE)
        )  THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' is not > ' || `minExclusive` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
where
    `minExclusive` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'MaxExclusiveConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) < TRY_CAST(`maxExclusive` AS DOUBLE)
            )
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) < TRY_CAST(`maxExclusive` AS DOUBLE)
            )
        )  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) < TRY_CAST(`maxExclusive` AS DOUBLE)
            )
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' not comparable with ' || `maxExclusive` || '.' 
        WHEN typ IS NOT NULL
        AND attr_typ IS NOT NULL
        AND NOT (
            TRY_CAST(val as DOUBLE) < TRY_CAST(`maxExclusive` as DOUBLE)
        )  THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' is not < ' || `maxExclusive` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
where
    `maxExclusive` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'MaxInclusiveConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) <= TRY_CAST(`maxInclusive` AS DOUBLE)
            )
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) <= TRY_CAST(`maxInclusive` AS DOUBLE)
            )
        )  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) <= TRY_CAST(`maxInclusive` AS DOUBLE)
            )
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' not comparable with ' || `maxInclusive` || '.' 
        WHEN typ IS NOT NULL
        AND attr_typ IS NOT NULL
        AND NOT (
            TRY_CAST(val as DOUBLE) <= TRY_CAST(`maxInclusive` as DOUBLE)
        )  THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' is not <= ' || `maxInclusive` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
where
    `maxInclusive` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'MinInclusiveConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) >= TRY_CAST(`minInclusive` AS DOUBLE)
            )
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) >= TRY_CAST(`minInclusive` AS DOUBLE)
            )
        )  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND (
            TRY_CAST(val AS DOUBLE) is NULL
            or NOT (
                TRY_CAST(val as DOUBLE) >= TRY_CAST(`minInclusive` AS DOUBLE)
            )
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' not comparable with ' || `minInclusive` || '.' 
        WHEN typ IS NOT NULL
        AND attr_typ IS NOT NULL
        AND NOT (
            TRY_CAST(val as DOUBLE) >= TRY_CAST(`minInclusive` as DOUBLE)
        )  THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' is not >= ' || `minInclusive` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
where
    `minInclusive` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'InConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND NOT ',' || `ins` || ',' LIKE '%,\"' || replace(val, '\"', '\\\"') || '\",%'  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND NOT ',' || `ins` || ',' LIKE '%,\"' || replace(val, '\"', '\\\"') || '\",%'  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND NOT ',' || `ins` || ',' LIKE '%,\"' || replace(val, '\"', '\\\"') || '\",%'   THEN 'Model validation for Property propertyPath failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' is not allowed.' 
        ELSE 'All ok'
    END as `text`  FROM A1
where
    `ins` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'PatternConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND NOT REGEXP(val, `pattern`)  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND NOT REGEXP(val, `pattern`)  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND NOT REGEXP(val, `pattern`)   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Value ' || IFNULL(val, 'NULL') || ' does not match pattern ' || `pattern` 
        ELSE 'All ok'
    END as `text`  FROM A1
WHERE
    `pattern` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'CountConstraintComponent(' || `parentPath` || `propertyPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN  NOT edeleted
        AND (
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    and attr_typ IS NOT NULL THEN 1
                    ELSE 0
                END
            ) > TRY_CAST(`maxCount` AS INTEGER) 
            OR SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    and attr_typ is NOT NULL THEN 1
                    ELSE 0
                END
            ) < TRY_CAST(`minCount` AS INTEGER)
        )  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN  NOT edeleted
        AND (
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    and attr_typ IS NOT NULL THEN 1
                    ELSE 0
                END
            ) > TRY_CAST(`maxCount` AS INTEGER) 
            OR SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    and attr_typ is NOT NULL THEN 1
                    ELSE 0
                END
            ) < TRY_CAST(`minCount` AS INTEGER)
        )   THEN `severity` 
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN  NOT edeleted
        AND (
            SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    and attr_typ IS NOT NULL THEN 1
                    ELSE 0
                END
            ) > TRY_CAST(`maxCount` AS INTEGER) 
            OR SUM(
                CASE
                    WHEN NOT COALESCE(adeleted, FALSE)
                    and attr_typ is NOT NULL THEN 1
                    ELSE 0
                END
            ) < TRY_CAST(`minCount` AS INTEGER)
        )   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '.  Found ' ||  TRY_CAST(
            count(
                CASE
                    WHEN NOT `adeleted` THEN 1
                    ELSE 0
                END
            ) AS STRING
        ) || ' relationships instead of\n                            [' || IFNULL('[' || `minCount`, '[0') || IFNULL(`maxCount` || ']', '[') || '!' 
        ELSE 'All ok'
    END as `text`  FROM A1
WHERE
    `minCount` is NOT NULL
    or `maxCount` is NOT NULL group by this,
    typ,
    propertyPath,
    minCount,
    maxCount,
    severity,
    edeleted,
    constraint_id,
    parentPath UNION ALL  SELECT this AS resource,
     'MinLengthConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND CHAR_LENGTH(val) < TRY_CAST(`minLength` AS INTEGER)  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND CHAR_LENGTH(val) < TRY_CAST(`minLength` AS INTEGER)  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND CHAR_LENGTH(val) < TRY_CAST(`minLength` AS INTEGER)   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Length of ' || IFNULL(val, 'NULL') || ' is < ' || `minLength` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
WHERE
    `minLength` IS NOT NULL
    and `index` IS NOT NULL UNION ALL  SELECT this AS resource,
     'MaxLengthConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND CHAR_LENGTH(val) > TRY_CAST(`maxLength` AS INTEGER)  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND CHAR_LENGTH(val) > TRY_CAST(`maxLength` AS INTEGER)  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND CHAR_LENGTH(val) > TRY_CAST(`maxLength` AS INTEGER)   THEN 'Model validation for Property ' || `propertyPath` || ' failed for ' || this || '. Length of ' || IFNULL(val, 'NULL') || ' is > ' || `maxLength` || '.' 
        ELSE 'All ok'
    END as `text`  FROM A1
WHERE
    `maxLength` IS NOT NULL
    and `index` IS NOT NULL UNION ALL   SELECT this AS resource,
     'DatatypeConstraintComponent(' || `parentPath` || `printPath` || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND  CASE
            WHEN (
                (
                    datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#string%' 
                )  
                AND (
                    (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                        AND  REGEXP(
                            val,
                            '^(?=.*[\\.eE])[+-]?(?:\\d+(?:\\.\\d*)?|\\.\\d+)(?:[eE][+-]?\\d+)?$'
                        )
                    ) 
                    OR (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                        AND  REGEXP(val, '^[+-]?\\d+$')
                    ) 
                    OR (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                        AND  REGEXP(val, '^(?i:true|false)$')
                    ) 
                    OR valueType LIKE '%' || datatypes || '%' 
                ) 
                OR (
                    propertyNodetype = '@json'
                    AND CASE
                        WHEN `val` IS JSON OBJECT THEN TRUE
                        ELSE FALSE
                    END
                ) 
                OR (
                    propertyNodetype = '@list'
                    AND `val` IS JSON ARRAY
                ) 
                OR (
                    propertyNodetype = '@value'
                    AND (
                        datatypes IS NULL
                        or NOT (
                            datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#string%' 
                        ) 
                    )
                ) 
            ) THEN false
            ELSE true
        END  THEN true
        ELSE false
    END as triggered,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND  CASE
            WHEN (
                (
                    datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#string%' 
                )  
                AND (
                    (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                        AND  REGEXP(
                            val,
                            '^(?=.*[\\.eE])[+-]?(?:\\d+(?:\\.\\d*)?|\\.\\d+)(?:[eE][+-]?\\d+)?$'
                        )
                    ) 
                    OR (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                        AND  REGEXP(val, '^[+-]?\\d+$')
                    ) 
                    OR (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                        AND  REGEXP(val, '^(?i:true|false)$')
                    ) 
                    OR valueType LIKE '%' || datatypes || '%' 
                ) 
                OR (
                    propertyNodetype = '@json'
                    AND CASE
                        WHEN `val` IS JSON OBJECT THEN TRUE
                        ELSE FALSE
                    END
                ) 
                OR (
                    propertyNodetype = '@list'
                    AND `val` IS JSON ARRAY
                ) 
                OR (
                    propertyNodetype = '@value'
                    AND (
                        datatypes IS NULL
                        or NOT (
                            datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#string%' 
                        ) 
                    )
                ) 
            ) THEN false
            ELSE true
        END  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        WHEN NOT edeleted
        AND attr_typ IS NOT NULL
        AND  CASE
            WHEN (
                (
                    datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                    OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#string%' 
                )  
                AND (
                    (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                        AND  REGEXP(
                            val,
                            '^(?=.*[\\.eE])[+-]?(?:\\d+(?:\\.\\d*)?|\\.\\d+)(?:[eE][+-]?\\d+)?$'
                        )
                    ) 
                    OR (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                        AND  REGEXP(val, '^[+-]?\\d+$')
                    ) 
                    OR (
                        datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                        AND  REGEXP(val, '^(?i:true|false)$')
                    ) 
                    OR valueType LIKE '%' || datatypes || '%' 
                ) 
                OR (
                    propertyNodetype = '@json'
                    AND CASE
                        WHEN `val` IS JSON OBJECT THEN TRUE
                        ELSE FALSE
                    END
                ) 
                OR (
                    propertyNodetype = '@list'
                    AND `val` IS JSON ARRAY
                ) 
                OR (
                    propertyNodetype = '@value'
                    AND (
                        datatypes IS NULL
                        or NOT (
                            datatypes LIKE '%http://www.w3.org/2001/XMLSchema#double%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#boolean%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#integer%'
                            OR  datatypes LIKE '%http://www.w3.org/2001/XMLSchema#string%' 
                        ) 
                    )
                ) 
            ) THEN false
            ELSE true
        END   THEN 'Datatype check failed. ' || CASE
            WHEN `datatypes` IS NULL THEN '\"' || `val` || '\" does not fit to '
            ELSE '\"' || `val`  || '\" does not fit to datatypes \"'  || `datatypes`  || '\" or '
        END  || 'property node type \"'  || `propertyNodetype`  || '\".' 
        ELSE 'All ok'
    END as `text`  FROM A1
where
    `index` IS NOT NULL
    AND `propertyNodetype` IN ('@value', '@list', '@json') UNION ALL  
SELECT
    this AS resource,
     'HasValueConstraintComponent('  || parentPath  || propertyPath  || ')' AS event,
     `constraint_id` as constraint_id,
     CASE
        WHEN 
        /* only non-deleted entities with a value present */
         NOT edeleted 
        AND attr_typ IS NOT NULL 
        /* and the actual value <> the required hasValue constant */
        
        AND CASE
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#double' THEN TRY_CAST(val as DOUBLE) <> TRY_CAST(hasValue as DOUBLE) 
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#integer' THEN TRY_CAST(val as INTEGER) <> TRY_CAST(hasValue as INTEGER) 
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#boolean' THEN TRY_CAST(val as BOOLEAN) <> TRY_CAST(hasValue as BOOLEAN) 
            ELSE `val` <> `hasValue`
        END  THEN TRUE
        ELSE FALSE
    END AS triggered,
     CASE
        WHEN 
        /* only non-deleted entities with a value present */
         NOT edeleted 
        AND attr_typ IS NOT NULL 
        /* and the actual value <> the required hasValue constant */
        
        AND CASE
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#double' THEN TRY_CAST(val as DOUBLE) <> TRY_CAST(hasValue as DOUBLE) 
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#integer' THEN TRY_CAST(val as INTEGER) <> TRY_CAST(hasValue as INTEGER) 
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#boolean' THEN TRY_CAST(val as BOOLEAN) <> TRY_CAST(hasValue as BOOLEAN) 
            ELSE `val` <> `hasValue`
        END  THEN `severity`
        ELSE 'ok'
    END AS severity,
     CASE
        
        WHEN 
        /* only non-deleted entities with a value present */
         NOT edeleted 
        AND attr_typ IS NOT NULL 
        /* and the actual value <> the required hasValue constant */
        
        AND CASE
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#double' THEN TRY_CAST(val as DOUBLE) <> TRY_CAST(hasValue as DOUBLE) 
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#integer' THEN TRY_CAST(val as INTEGER) <> TRY_CAST(hasValue as INTEGER) 
            WHEN `valueType` = 'http://www.w3.org/2001/XMLSchema#boolean' THEN TRY_CAST(val as BOOLEAN) <> TRY_CAST(hasValue as BOOLEAN) 
            ELSE `val` <> `hasValue`
        END   THEN  'Model validation for Property '  || propertyPath  || ' failed for '  || this  || '. Value \"'  || val  || '\" does not match required \"'  || hasValue  || '\".' 
        ELSE 'All ok' 
    END AS text  FROM A1  WHERE hasValue IS NOT NULL
    AND `index` IS NOT NULL;

In [ ]:
%%flink_gateway_sql_query
INSERT INTO
    constraint_trigger_table WITH  needed AS (
        
        SELECT
             target_constraint_id,
             COUNT(*) AS needed_count 
        FROM
             constraint_combination_table 
        WHERE
             operation = 'OR' 
        GROUP BY
             target_constraint_id 
    ),
      fired AS (
        
        SELECT
            DISTINCT  t.resource,
             comb.target_constraint_id,
             nm.needed_count,
             COUNT(DISTINCT t.constraint_id) AS fired_count,
              LISTAGG(DISTINCT t.event, ' AND ') AS events,
             LISTAGG(DISTINCT t.text, ' AND ') AS texts  
        FROM
            (
                
                SELECT
                    * 
                FROM
                    constraint_trigger_table 
                WHERE
                    triggered = TRUE 
                ORDER BY
                    event,
                    text 
            ) AS t 
            JOIN  constraint_combination_table AS comb  ON comb.member_constraint_id = t.constraint_id 
            AND comb.operation = 'OR' 
            JOIN  needed AS nm  ON nm.target_constraint_id = comb.target_constraint_id 
        WHERE
             t.triggered = TRUE 
        GROUP BY
             t.resource,
             comb.target_constraint_id,
             nm.needed_count 
        HAVING
             COUNT(DISTINCT t.constraint_id) = nm.needed_count  
    )  SELECT  f.resource AS resource,
     f.events AS event,
     f.target_constraint_id AS constraint_id,
     TRUE AS triggered,
     ct.severity AS severity,
     'OR rule ' || CAST(f.target_constraint_id as STRING)  || ' fired (' || f.texts || ')' AS text  FROM  fired AS f JOIN  constraint_table AS ct  ON ct.id = f.target_constraint_id ;

In [ ]:
%%flink_gateway_sql_query
INSERT INTO
    alerts_bulk SELECT  t.resource,
     t.event,
     'Development' AS environment,
      ARRAY ['SHACL Validator'] AS service,
       CASE
        
        WHEN MAX(
            CASE
                WHEN t.triggered THEN 1
                ELSE 0
            END
        ) = 1  THEN MAX(t.severity) 
        ELSE 'ok' 
    END AS severity,
     'customer' AS customer,
     CASE
        
        WHEN MAX(
            CASE
                WHEN t.triggered THEN 1
                ELSE 0
            END
        ) = 1  THEN MAX(t.text) 
        ELSE 'ok' 
    END AS text  FROM  constraint_trigger_table AS t 
    JOIN constraint_combination_table AS comb  ON comb.operation = 'PUBLISH' 
    AND comb.member_constraint_id = t.constraint_id 
    JOIN constraint_table AS ct  ON ct.id = comb.member_constraint_id GROUP BY  t.resource,
     t.event;